$\newcommand{\Sum}{\sum\limits}$
$\newcommand{\Prod}{\prod\limits}$
$\newcommand{\Max}{\max\limits}$
$\newcommand{\Min}{\min\limits}$
$\newcommand{\Int}{\int\limits}$
$\newcommand{\Exp}{\mathbb{E}}$
$\newcommand{\Var}{\mathbb{V}}$
$\newcommand{\Energy}{\mathcal{E}}$
$\newcommand{\Prob}{\mathcal{P}}$
$\newcommand{\N}{\mathcal{N}}$

$\newcommand{\LogLike}{\mathcal{L}}$
$\newcommand{\Like}{\ell}$

$\newcommand{\bolda}{\boldsymbol{a}}$
$\newcommand{\boldA}{\boldsymbol{A}}$
$\newcommand{\ba}{\bolda}$
$\newcommand{\bA}{\boldA}$

$\newcommand{\boldb}{\boldsymbol{b}}$
$\newcommand{\boldB}{\boldsymbol{B}}$
$\newcommand{\bb}{\boldb}$
$\newcommand{\bB}{\boldB}$

$\newcommand{\boldd}{\boldsymbol{d}}$
$\newcommand{\boldD}{\boldsymbol{D}}$
$\newcommand{\bd}{\boldd}$
$\newcommand{\bD}{\boldED}$
$\newcommand{\bolde}{\boldsymbol{e}}$
$\newcommand{\boldE}{\boldsymbol{E}}$
$\newcommand{\be}{\bolde}$
$\newcommand{\bE}{\boldE}$
$\newcommand{\boldf}{\boldsymbol{f}}$
$\newcommand{\boldF}{\boldsymbol{F}}$
$\newcommand{\bf}{\boldf}$
$\newcommand{\bF}{\boldF}$
$\newcommand{\bolds}{\boldsymbol{s}}$
$\newcommand{\boldS}{\boldsymbol{S}}$
$\newcommand{\bs}{\boldsymbol{\bolds}}$
$\newcommand{\bS}{\boldsymbol{\boldS}}$
$\newcommand{\boldt}{\boldsymbol{s}}$
$\newcommand{\boldT}{\boldsymbol{S}}$
$\newcommand{\bt}{\boldsymbol{\boldt}}$
$\newcommand{\bT}{\boldsymbol{\boldT}}$

$\newcommand{\boldu}{\boldsymbol{u}}$
$\newcommand{\boldU}{\boldsymbol{U}}$
$\newcommand{\bu}{\boldu}$
$\newcommand{\bU}{\boldU}$
$\newcommand{\boldv}{\boldsymbol{v}}$
$\newcommand{\boldV}{\boldsymbol{V}}$
$\newcommand{\bv}{\boldv}$
$\newcommand{\bV}{\boldV}$
$\newcommand{\boldx}{\boldsymbol{x}}$
$\newcommand{\boldX}{\boldsymbol{X}}$
$\newcommand{\bx}{\boldx}$
$\newcommand{\bX}{\boldX}$
$\newcommand{\boldY}{\boldsymbol{Y}}$
$\newcommand{\boldy}{\boldsymbol{y}}$
$\newcommand{\bY}{\boldY}$
$\newcommand{\by}{\boldy}$
$\newcommand{\boldZ}{\boldsymbol{Z}}$
$\newcommand{\boldz}{\boldsymbol{z}}$
$\newcommand{\bZ}{\boldZ}$
$\newcommand{\bz}{\boldz}$

$\newcommand{\boldTheta}{\boldsymbol{\Theta}}$
$\newcommand{\boldtheta}{\boldsymbol{\theta}}$
$\newcommand{\bTheta}{\boldTheta}$
$\newcommand{\btheta}{\boldtheta}$

$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\diag}{\text{diag}}$

<a id='toc'></a>
# Содержание
* [Теория](#theory)
    * [PCA (Principal Component Analysis)](#pca)
    * [Probabilistic PCA](#probabilistic)
* [Методы нахождения собственных векторов](#methods)
* [Применение](#application)
* []

# Теория

Хотим выбрать плоскость такую, что в перпендикулярном ей направлении дисперсия значений проекций точек максимальна. Пусть $u$ --- перпендикуляр к плоскости. Пусть $F(\boldu;\boldX)$ --- среднее значение квадрата проекций точек на направление $\boldu$:
\begin{gather}
F(\boldu;\boldX) = \frac{1}{N}\Sum_{i=1}^N \left\|\frac{(\boldx_i - \bar{\boldx}, \boldu)}{(\boldu, \boldu)} \boldu\right\|^2_2 = 
\frac{1}{N}\frac{\Sum_{i=1}^N (\boldx_i - \bar{\boldx}, \boldu)^2}{(\boldu, \boldu)} \rightarrow \max_{\boldu \in \mathbb{R}^N} 
\end{gather}
Пусть вектор $\boldu$ нормирован. Тогда приходим к задаче оптимизации
\begin{gather}
F(\boldu;\boldX)  = \frac{1}{N}\Sum_{i=1}^N (\boldx_i - \bar{\boldx}, \boldu)^2 \rightarrow \max_{\|\boldu\|^2_2 = 1} 
\end{gather}

Лагранжиан имеет вид
\begin{gather}
L(\boldu,\lambda) = \frac{1}{N}\Sum_{i=1}^N (\boldx_i - \bar{\boldx}, \boldu)^2 - \frac{\lambda}{2} (\|\boldu\|_2^2 - 1)
\end{gather}
Дифференцируя $L(\boldu, \lambda)$ по $\boldu$, получаем
\begin{gather}
\frac{\partial L(\boldu, \lambda)}{\partial \lambda} = \frac{2}{N}(\boldx_i - \bar{\boldx}, \boldu) \cdot (\boldx_i - \bar{\boldx}) - 2\lambda \boldu \\
\frac{\partial L(\boldu, \lambda)}{\partial \lambda}  = 0 \Leftrightarrow \lambda \boldu = \frac{1}{N}\Sum_{i=1}^N (\boldx_i - \bar{\boldx}) \cdot (\boldx_i - \bar{\boldx}, \boldu) = \frac{1}{N}\Sum_{i=1}^N (\boldx_i - \bar{\boldx}) (\boldx_i - \bar{\boldx})^T \cdot \boldu
\end{gather}
Здесь появляется матрица ковариации $\boldS$:
\begin{gather}
\boldS = \frac{1}{N}\Sum_{i=1}^N (\boldx_i - \bar{\boldx}) (\boldx_i - \bar{\boldx})^T
\end{gather}

Таким образом, искомое направление задается собственный вектором матрицы ковариаций $\boldS$. При этом среднее значение квадратов длин проекций равно собственному значению.
\begin{gather}
F(\boldu;\boldX) = \frac{1}{N}\Sum_{i=1}^N (\boldx_i - \bar{\boldx}, \boldu)^2 = \frac{1}{N}\Sum_{i=1}^N \boldu^T (\boldx_i - \bar{\boldx})(\boldx_i - \bar{\boldx})^T \boldu = \boldu^T \boldS \boldu = \lambda \|\boldu\|_2^2 = \lambda.
\end{gather}
Выбрав собственный вектор $\boldu_1$, которому соответствует максимальное собственное значение $\lambda_1$ матрицы ковариаций $\boldS$, получим искомое направление.

#### Итеративная процедура нахождения направлений максимальной дисперсии

Пусть $\lambda_1, \lambda_2, \dots, \lambda_D$ --- собственные векторы матрицы $S$, в порядке убывания, а $\boldu_1, \boldu_2, \dots, \boldu_D$ --- соответствующие им собственные значения. Докажем, что описанная выше процедура итеративного нахождения направлений наибольшей дисперсии приведт в конечном счете к векторам  $\boldu_1, \boldu_2, \dots, \boldu_D$.

На первом этапе $\boldS_1 = \boldS$ получили вектор направления $\boldu_1$ с собственным значением $\lambda_1$. Теперь найдем следующий вектор. Выясним, как будет выглядеть матрица ковариаций на второй итерации $\boldS_2$. 

Вычтем из каждого вектора $\boldx_i$ его проекцию на $\boldu_1$ и для полученных векторов найдем матрицу ковариации.
\begin{gather}
\boldS_2 = \frac{1}{N}\Sum_{i=1}^N\left(\boldx_i - (\boldx_i, \boldu_1)\boldu_1 - (\bar{\boldx} - (\bar{\boldx}, \boldu_1)\boldu_1)\right)\left(\boldx_i - (\boldx_i, \boldu_1)\boldu_1 - (\bar{\boldx} - (\bar{\boldx}, \boldu_1)\boldu_1)\right)^T = \\ = \frac{1}{N}\Sum_{i=1}^N\left(\boldx_i - \bar{\boldx} - (\boldx_i - \bar{\boldx}, \boldu_1)\boldu_1 \right) \left(\boldx_i - \bar{\boldx} - (\boldx_i - \bar{\boldx}, \boldu_1)\boldu_1 \right)^T = \\ =\frac{1}{N} \Sum_{i=1}^N \left\{(\boldx_i - \bar{\boldx})(\boldx_i - \bar{\boldx})^T - 2 (\boldx_i - \bar{\boldx}) \cdot (\boldx_i - \bar{\boldx}, \boldu_1)  \cdot \boldu_1^T + \boldu_1 \boldu_1^T (\boldx_i - \bar{\boldx}, \boldu_1)^2 \right\} = \\ =  \frac{1}{N}\Sum_{i=1}^N \left\{(\boldx_i - \bar{\boldx})(\boldx_i - \bar{\boldx})^T - \boldu_1\boldu_1^T (\boldx_i - \bar{\boldx})(\boldx_i - \bar{\boldx})^T \right\} = \boldS_1(\boldE - \boldu_1\boldu_1^T) = (\boldE - \boldu_1\boldu_1^T) \boldS_1.
\end{gather}

Заметим, что собственные значения $\lambda_2 \ge \dots \ge \lambda_D \ge 0$ матрицы $\boldS$ являются также собственными значениями матрицы ковариаций $\boldS_2$. Действительно,
$$
\boldS_2 \boldu_2 = (\boldE - \boldu_1\boldu_1^T) \boldS_1 \boldu_2 = \lambda_2(\boldE - \boldu_1\boldu_1^T) \boldu_2 =  \lambda_2\boldu_2 - \boldu_1\boldu_1^T \boldu_2 = \lambda_2\boldu_2.
$$
Здесь учтено, что векторы $\boldu_1, \dots, \boldu_D$ ортогональны.

Заметим что $\lambda_2, \dots, \lambda_D$ --- это первые $D-1$ собственных значений. Последнее дополнительное собственное значение $\boldS_2$ равно нулю. Ему соответствует собственный вектор $\boldu_1$:
$$
S_2\boldu_1 = (1-\boldu_1\boldu_1^T)\boldS_1 \boldu_1 = \lambda_1 (1-\boldu_1\boldu_1^T) \boldu_1 = \lambda_1(\boldu_1 - \boldu_1) = 0.
$$

Таким образом, на втором этапе итеративной процедуры будет выбрано направление $\boldu_2$, так как ему соответствует максимальная дисперсия $\lambda_2$. В общем случае на $i$-ом этапе будет выбран вектор $\boldu_i$ с дисперсией $\lambda_i$. Что и требовалось доказать.

Смылс доказанного выше сводится к тому, что в итеративной процедуре как таковой нет смысла: достаточно найти собственные вектора и собственные значения для исходной матрицы ковариаций.

#### Смысл проекции
* Собственные значения --- дисперсия точек вдоль направления соответствующего собственного вектора
* Если $M > 0$ собственных значений равны нулю или близки к нему, то данные сосредоточены в подпространстве размерности $D - M$
* Достаточно выбрать только первые $K$ собственных векторов, соответствующих максимальным собственным значениям.

# Сингулярное разложение

Пусть дана матрица $X \in \RR^{m\times n}$. В задачах машинного обучения это матрица объекты-признаки. В данном случае положим, что $m$ --- число признаков, а $n$ --- число объектов, т.е. признаки записаны в столбцы, а не строки.  

Требуется спроектировать векторы $\boldx_1, \dots, \boldx_n$ из пространства размерности $m$ на некоторое его $k$-мерное подпространство, так чтобы сумма квадратов длин проекций была максимальна. 

Для всякой матрицы $X \in \RR^{m\times n}$ существует сингулярное разложение:
$$
X = U \Sigma V^T,
$$
где $U \in \RR^{m\times m}$, $V \in \RR^{n\times n}$, $\Sigma = \text{diag}^*_{m,n}[\sigma_1, \dots, \sigma_{\min(m,n)}] \in \RR^{m\times n}$. Матрицы $U$ и $V$ --- ортогональные матрицы, матрица $\Sigma$ --- псевдоортогональная. Пусть для определенности $m \le n$ (такой случай наиболее характерен на парктике).

* Матрица $U = [\boldu_1, \dots, \boldu_m]$ --- матрица собственных векторов отображения $XX^T$:
\begin{gather}
XX^T = U\Sigma V^T \cdot V \Sigma^T U^T = U\Sigma \Sigma^T U^T = U ~\underbrace{\diag[\sigma_1^2, \dots, \sigma_m^2]}_{m\times m}U^T\\
XX^T U = U \diag[\sigma_1^2, \dots, \sigma_m^2] U^T U = [\sigma_1^2 \boldu_1, \dots, \sigma_m^2 \boldu_m].
\end{gather}
* Матрица $V = [\boldv_1, \dots, \boldv_n]$ --- матрица собственных векторов отображения $X^TX$:
\begin{gather}
X^TX = V\Sigma^T U^T \cdot U \Sigma V^T = U\Sigma \Sigma^T U^T = V ~\underbrace{\diag[\sigma_1^2, \dots, \sigma_m^2, 0, \dots, 0]}_{n\times n}V^T\\
X^TXV = V ~\diag[\sigma_1^2, \dots, \sigma_m^2, 0, \dots, 0]V^T V = [\sigma_1^2\boldv_1, \dots,\sigma_m^2 \boldv_m, 0, \dots, 0]
\end{gather}

Определив матрицы $U$ и $\Sigma$, можем затем найти матрицу $V$.
\begin{gather}
XV = U\Sigma V^T V = U \Sigma = [\sigma_1\boldu_1, \dots, \sigma_m\boldu_m, 0, \dots, 0]
\end{gather}
Заметим, что если все $m$ собственных значений $\sigma_1^2$ матрицы $XX^T$ (или $X^TX$) отличны от нуля, то
выше будут найдены все собственные векторы. В противном случае останется вопрос о нахождении собственных векторов матрицы $XX^T$ для нулевых собственных значений. Но такие векторы нам в дальнейшем не потребуются, поэтому этот вопрос мы опустим.

Аналогично, определив матрицы $V$ и $\Sigma$, можем затем найти матрицу $U$.
\begin{gather}
X^T U = V\Sigma^T U^T U = V \Sigma^T = [\sigma_1\boldv_1, \dots, \sigma_m\boldv_m]
\end{gather}
В данном случае также как и ранее находим все собственные векторы матрицы $X^TX$ с ненулевыми собственными значениями.

TODO

Таким образом, для нахождения сингулярного разложения достаточно найти собственные векторы и собственные значения матрицы $XX^T$ или $X^TX$ (в зависимости от того, какая из задач проще). В нашем случае выгодно рассматривать матрицу $XX^T$. 
* Нахождение $m$ собственных векторов $U$ матрицы $XX^T \in \RR^{m \times m}$ и соответствующих собственных значений $\sigma_1 \ge \dots \ge \sigma_m \ge 0$. Пусть число ненулевых собственных значений равно $r$: $\sigma_1 \ge \dots \ge \sigma_r > \sigma_{r+1} = \dots \sigma_m = 0$.
* Нахождение $m - r$ собственных векторов матрицы $XX^T$ с нулевыми собственными значениями.
* Нахождение первых $r$ собственных векторов $V$ матрицы $X^TX \in \RR^{n \times n}$, соответствующих собственным значениям $\sigma_1 \ge \dots \ge \sigma_m \ge 0$:
    $$
    X^T U = [\sigma_1\boldv_1, \dots, \sigma_r\boldv_r, 0, \dots, 0].
    $$
* Нахождение оставшихся $n - r$ собственных значений матрицы $X^TX$ с нулевыми собственными значениями. TODO

<a id='methods'></a>
# Методы нахождения собственных значений<sup>[toc](#toc)</sup>

## Итерационная процедура нахождения собственных значений [[toc](#toc)]

$$\|A - X\|_2^2 \rightarrow \min_{X \in \RR^{m \times n}}$$.
В такой постановке ответ очевиден --- это просто $X = A$. Поэтому наложим дополнительное ограничение $\rank X = 1$. Пусть 
$A = U\Sigma V^T$. Тогда $X = U X_1 V^T$ [TODO why].
$$
\|A - X\|_2^2 = \|U(\Sigma - X_1)V^T\|_2^2 = \|\Sigma - X_1\|_2^2.
$$
В таком случае $X_1$ --- это просто матрица с единственным ненулевым элементом, равным $\sigma_1$. При этом
$X = UX_1V^T = \sigma_1 \boldu_1 \boldv_1^T$. 

Обозначим $X = \boldx \boldy^T$ и будем решать оптимизационную задачу
$$
\frac{1}{2}\Sum_{i,j}(a_{i,j} -x_i y_j)^2 \rightarrow \min.
$$
В таком случае 
\begin{gather}
x_i = \frac{\Sum_j a_{i,j}y_j}{\Sum_j y_j^2}, \qquad y_j = \frac{\Sum_i a_{i,j} x_i}{\Sum_i x^2_i}
\end{gather}

In [1]:
import numpy as np

In [3]:

np.linalg.norm

<function numpy.linalg.linalg.norm>

In [23]:
import numpy as np

def find_eigenvalue(A):
    A = A.copy()
    eig_vals = []
    eig_vecs_left = []
    eig_vecs_right = []
    n_eig = 0
    while 1:
        x = np.random.rand(A.shape[0])
        y = np.random.rand(A.shape[1])
        Fs = [np.linalg.norm(A - x[:, None] * y[None, :], 'fro')]
        delta = np.inf
        n_iter = 0
        while np.abs(delta) > 1e-12:
            x = np.sum(A * y[None, :], axis=1) / np.sum(y**2)
            y = np.sum(A * x[:, None], axis=0) / np.sum(x**2)
            F = np.linalg.norm(A - x[:, None] * y[None, :], 'fro')
            delta = Fs[-1] - F
            Fs.append(F)
            print('n_eig = {}, n_iter = {}: delta = {}.'.format(n_eig, n_iter, delta))
            n_iter += 1
        eig_val = np.sum(x**2) * np.sum(y**2)
        if eig_val > 1e-10:
            eig_vals.append(eig_val)
            eig_vecs_left.append(x[:, None])
            eig_vecs_right.append(y[:, None])
            n_eig += 1
            if n_eig == min(A.shape):
                print('All eigenvalues was found.')
                break
            A -= x[:, None] * y[None, :]
        else:
            break
    return eig_vals, eig_vecs_left, eig_vecs_right

In [24]:
np.random.seed(0)
A = np.random.rand(3, 5)
print(A)
eig_vals, eig_vecs_left, eig_vecs_right = find_eigenvalue(A)
print('my eigenvalues:', eig_vals)
print('np eigenvalues:', np.linalg.eigvals(np.dot(A, A.T)))

[[ 0.5488135   0.71518937  0.60276338  0.54488318  0.4236548 ]
 [ 0.64589411  0.43758721  0.891773    0.96366276  0.38344152]
 [ 0.79172504  0.52889492  0.56804456  0.92559664  0.07103606]]
n_eig = 0, n_iter = 0: delta = 1.464341909284908.
n_eig = 0, n_iter = 1: delta = 0.0048035466556891615.
n_eig = 0, n_iter = 2: delta = 3.849789232068801e-06.
n_eig = 0, n_iter = 3: delta = 3.0807141282451767e-09.
n_eig = 0, n_iter = 4: delta = 2.4685808952540356e-12.
n_eig = 0, n_iter = 5: delta = 1.9984014443252818e-15.
n_eig = 1, n_iter = 0: delta = 0.935611644668977.
n_eig = 1, n_iter = 1: delta = 0.007029025868303396.
n_eig = 1, n_iter = 2: delta = 0.002573123895356355.
n_eig = 1, n_iter = 3: delta = 0.0008794256265877709.
n_eig = 1, n_iter = 4: delta = 0.0002934113825127649.
n_eig = 1, n_iter = 5: delta = 9.710293016712557e-05.
n_eig = 1, n_iter = 6: delta = 3.2049282698454196e-05.
n_eig = 1, n_iter = 7: delta = 1.0568611270411754e-05.
n_eig = 1, n_iter = 8: delta = 3.48409574524311e-06.
n_eig 

In [18]:
2.44 ** 2

59536

<a id='application'></a>
# Применение PCA<sup>[toc](#toc)</sup>

<a id='app_images'></a>
## Применение к сжатию изображений<sup>[toc](#toc)</sup>

<a id='app_recover'></a>
## Задача восстановления профиля крыла<sup>[toc](#toc)</sup>